In [ ]:
!pip install transformers
!pip install torchtext==0.8.0 --quiet
!pip install torch==1.7.1 --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchtext
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

from transformers import BertTokenizer, BertModel

import numpy as np

import time
import random
import functools

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

dir = 'drive/MyDrive/BS/DATA_EXTRACTION/'
corp_cased = dir + 'corp_cased.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Установка seed для воспроизводимости результатов

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
MODE_RU = True #True - rus, False - multilanguage

# Подготовка данных

Импортируем токенизатор для BERT. Он определяет как текст в модели должен обрабатываться. Также он содержит словарь предобученной модели BERT. К сожалению, для русского языка нет отдельной предобученной модели и токенизатор, поэтому будем использовать многоязычную \
`bert-base-multilingual-uncased`.

Чтобы использовать предобученную модель нужно, чтобы словарь в точности совпадал со словарем предобученной модели.

In [ ]:
if MODE_RU:
    tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
else:
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') 

Нужно убедиться, что входная последовательность форматирована так же, как и предусмотрено в BERT

BERT была обучена на последовательностях, начинающихся с токена `[CLS]`

Пример преобразования:

```python
text = ['Я', 'пошел', 'в', 'тот', 'магазин']
```

```python
text = ['[CLS]', 'Я', 'пошел', 'в', 'тот', 'магазин']
```

Необходимо дополнительно убедиться, что неизвестный токен обозначается как `[UNK]`, а также токен выравнивания длины (padding token) обозначается как `[PAD]`

Получим специальные токены:

In [ ]:
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, pad_token, unk_token)

[CLS] [PAD] [UNK]


Получим индексы специальных токенов с помощью `convert_tokens_to_ids`

In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, pad_token_idx, unk_token_idx)

101 0 100


Необходимо, чтобы длина предложений во входных последовательностях не превосходила максимальной длины последовательностей в предобученной модели

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']

print(max_input_length)

512


Определим две вспомогательные функции для словарей

Первая будет 'подрезать' входные последовательности токенов до максимальной желаемой длины. Далее токены будут преобразовываться в индексы с помощью словаря. Она будет использоваться для последовательностей, для которых мы хотим найти теги.

Важно отметить, что послеодовательности будут подрезаться до max_length-1 из-за начального токена `[CLS]`

In [ ]:
def cut_and_convert_to_id(tokens, tokenizer, max_input_length):
    tokens = tokens[:max_input_length-1]
    tokens = tokenizer.convert_tokens_to_ids(tokens)
    return tokens

Вторая вспомогательная функция просто подрезает последователньости до максимальной желаемой длины. Это используется для тегов. Мы не хотим передавать теги через словарь предобученной модели так как мы преимущественно используем русский язык. Будем строить словарь сами

In [ ]:
def cut_to_max_length(tokens, max_input_length):
    tokens = tokens[:max_input_length-1]
    return tokens

Мы используем библиотеку `functools`, чтобы передать в функцию аргументы.

In [ ]:
text_preprocessor = functools.partial(cut_and_convert_to_id,
                                      tokenizer = tokenizer,
                                      max_input_length = 64)

tag_preprocessor = functools.partial(cut_to_max_length,
                                     max_input_length = 64)

Дальше определим `Field`

Для поля `TEXT`, который будет обрабатывать последовательности, для которых нужно найти теги сделаем следующее:\
1) Не будем использовать словарь\
2) Приведем текст в нижний регистр, так как модель в нижнем регистре\
3) Предобработка будет проводиться функцией text_preprocessor\
4) Зададим индексы специальных токенов\

Для поля `UD_TAGS` нужно убедиться, что длина последовательности тегов соответствует длине текстовой последовательности. Так как мы добавляли `[CLS]` токен в начало текстовых последовательностей, то нужно проделать то же самое и с тегами. Добавим `<pad>` токен в начало и укажем модели не использовать его при подсчете метрик качества. У нас также не будет неизвестных тегов. Функция предобработки - `tag_preprocessor`.

In [ ]:
TEXT = Field(use_vocab = False,
                  lower = False,
                  preprocessing = text_preprocessor,
                  init_token = init_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

UD_TAGS = Field(unk_token = '<unk>',
                     init_token = '<pad>',
                     preprocessing = tag_preprocessor)

Определим какие из созданны полей соотвествуют полям в датасете

In [ ]:
fields = (("text", TEXT), ("udtags", UD_TAGS))

Загрузим датасет и сразу разделим его

In [ ]:
dataset = TabularDataset(path=corp_cased, format='tsv', fields=fields, skip_header=True)

In [ ]:
train_data, valid_data, test_data = dataset.split([0.8, 0.1, 0.1])

Пример

In [ ]:
print(vars(train_data.examples[0]))

{'text': [5453, 66814, 16781, 42755, 9620, 47602, 27409, 30694, 23530, 851, 24265, 61690, 24145, 12908], 'udtags': ['VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'INFN', 'NOUN', 'PRTF', 'NOUN', 'CONJ', 'INFN', 'NOUN', 'ADJF', 'NOUN']}


Нужно построить словарь тегов. Сделаем это с помощью `.build_vocab` на `train_data`.

In [ ]:
UD_TAGS.build_vocab(train_data)

print(UD_TAGS.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f1953b3bf90>>, {'<unk>': 0, '<pad>': 1, 'NOUN': 2, 'ADJF': 3, 'PREP': 4, 'VERB': 5, 'CONJ': 6, 'ADVB': 7, 'NPRO': 8, 'PRCL': 9, 'INFN': 10, 'UNKN': 11, 'PRTF': 12, 'Geox': 13, 'Name': 14, 'Surn': 15, 'ADJS': 16, 'PRTS': 17, 'NUMR': 18, 'GRND': 19, 'PRED': 20, 'COMP': 21, 'INTJ': 22, 'Patr': 23})


Определим итераторы. Они задают к порции данных будут подаваться во время обучения. Определим размер порции и также определим `device`, который автоматически будет подавать порции на GPU

BERT достаточно большая модель, поэтому размер порции сравнительно небольшой.

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    sort=False,
    batch_size = BATCH_SIZE,
    device = device)

# Построение модели

Модель довольно проста, все сложные моменты скрыты внутри BERT. Можем считать, что BERT - это такой эмбеддинг слой и все что нам нужно сделать - это добавить линейный слой

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bert.png?raw=1)

В прошлом, желтые квадратики были эмбеддингами, которые создал соответствующий слой, но сейчас это эмбеддинги, которые мы получаем от предобученной модели BERT. Все входные значения подаются в BERT в одно и то же время. Стрелки между эмбеддингами BERT указывают, что эмбеддинги не считаются на для каждого токена индивидуально, а основываются на других токенах последовательности, что создает контекст.

Обратим внимание, что мы не задаем `embedding_dim` для нашей модели. Это размер выхода из предобученной модели и мы не можем менять его. Таким образом, мы просто получаем эту размерность из атрибута модели `hidden_size`

In [ ]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
  
        #text = [sent len, batch size]
    
        text = text.permute(1, 0)
        
        #text = [batch size, sent len]
        
        embedded = self.dropout(self.bert(text)[0])
        
        #embedded = [batch size, seq len, emb dim]
                
        embedded = embedded.permute(1, 0, 2)
                    
        #embedded = [sent len, batch size, emb dim]
        
        predictions = self.fc(self.dropout(embedded))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions

Далее, загрузим предобученную модель - прежде мы загружали только токенизатор модели.

In [ ]:
if MODE_RU:
    bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
else:
    bert = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Обучение модели

Определим параметры для dropout слоя

In [ ]:
OUTPUT_DIM = len(UD_TAGS.vocab)
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)

Посчитаем количество обучаемых параметров. Включаются параметры линейного слоя и все параметры BERT

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 177,871,896 trainable parameters


Определим оптимизатор. Обычно, при использовании предобученной модели мы используем коэффициент скорости обучения меньше, чем обычно. Это делается потому что мы не хотим радикально менять параметры, так как это может вызвать забывание того, что она выучила. Этот феномен называется катастрофическое забывание (catastrophic forgetting)

Выбрали 5е-5, так как это одно из значений, рекомендованных создателями модели. Могут быть и лучшие значения.

In [ ]:
LEARNING_RATE = 5e-5

optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

Определим функцию потерь, убеждаясь, что игнорируем токены заполнения (pad)

In [ ]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

Дальше, просто помещаемс модель на GPU

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

Определим функцию, которая подсчитываем точность предсказанных тегов, игнорируя токены заполнения

Определим `train` и `evaluate` функции для обучения и тестирования модели

In [ ]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0

    model.train()

    all_preds = []
    all_tags = []
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.udtags
                
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        all_preds.append(predictions.detach().cpu().numpy())
        all_tags.append(tags.detach().cpu().numpy())
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()

        
    return epoch_loss / len(iterator), np.concatenate(all_preds, 0).argmax(1).reshape(-1), np.concatenate(all_tags, 0)

In [ ]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    
    model.eval()

    all_preds = []
    all_tags = []
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            all_preds.append(predictions.detach().cpu().numpy())
            all_tags.append(tags.detach().cpu().numpy())
            
            loss = criterion(predictions, tags)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), np.concatenate(all_preds, 0).argmax(1).reshape(-1), np.concatenate(all_tags, 0)

Определим вспомогаетльную функцию для подсчета времени

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
cr_labels = list(range(2, len(UD_TAGS.vocab.itos)))
cr_names = [UD_TAGS.vocab.itos[l] for l in cr_labels]

print(cr_labels)
print(cr_names)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
['NOUN', 'ADJF', 'PREP', 'VERB', 'CONJ', 'ADVB', 'NPRO', 'PRCL', 'INFN', 'UNKN', 'PRTF', 'Geox', 'Name', 'Surn', 'ADJS', 'PRTS', 'NUMR', 'GRND', 'PRED', 'COMP', 'INTJ', 'Patr']


Наконец, можем обучить модели.

Эта модель обучается достаточно долго из-за большого количества параметров

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_preds, train_tags = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, _, __ = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')

    print(classification_report(train_tags, train_preds, labels=cr_labels, target_names=cr_names))

    print(f'\t Val Loss: {valid_loss:.3f}')

Epoch: 01 | Epoch Time: 14m 39s
	Train Loss: 0.303
              precision    recall  f1-score   support

        NOUN       0.11      0.96      0.20    405318
        ADJF       0.86      0.92      0.89    185307
        PREP       0.97      0.98      0.98    148372
        VERB       0.81      0.92      0.86    110462
        CONJ       0.78      0.97      0.87    109946
        ADVB       0.44      0.87      0.59     46837
        NPRO       0.96      0.90      0.93     37182
        PRCL       0.87      0.92      0.89     37148
        INFN       0.93      0.91      0.92     28957
        UNKN       0.15      0.40      0.22     26973
        PRTF       0.76      0.58      0.66     19312
        Geox       0.83      0.84      0.84     17958
        Name       0.68      0.73      0.70     14785
        Surn       0.72      0.60      0.65     10881
        ADJS       0.82      0.63      0.72      9953
        PRTS       0.91      0.82      0.86      8518
        NUMR       0.94      0

Можем загрузить лучшую модель и попробовать ее на тестовом множестве

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_preds, test_tags = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f}')

print(test_preds[10:])
print(test_tags[10:])

print(classification_report(test_tags, test_preds, labels=cr_labels, target_names=cr_names))


Test Loss: 0.208
[ 2  2  2 ... 11  2  2]
[1 1 1 ... 2 1 1]
              precision    recall  f1-score   support

        NOUN       0.11      0.96      0.20     50104
        ADJF       0.91      0.94      0.93     23174
        PREP       0.99      0.99      0.99     18333
        VERB       0.91      0.94      0.93     13674
        CONJ       0.76      0.99      0.86     13610
        ADVB       0.94      0.90      0.92      5839
        NPRO       0.97      0.94      0.96      4536
        PRCL       0.96      0.96      0.96      4671
        INFN       0.93      0.94      0.93      3738
        UNKN       0.65      0.46      0.54      3401
        PRTF       0.77      0.72      0.75      2442
        Geox       0.94      0.91      0.92      2191
        Name       0.84      0.80      0.82      1818
        Surn       0.71      0.73      0.72      1310
        ADJS       0.84      0.78      0.81      1178
        PRTS       0.93      0.87      0.90      1029
        NUMR       0.9

# Вывод

Мы увидим как использовать модель для тегирования последовательностей.

Если мы подадим строку, то нужно ее разделить на индивидуальные токены. Мы сделаем это с помощью функции `tokenize` из `tokenizer`. После, перевести токены в числа так же, как мы делали раньше, используя `convert_tokens_to_ids`. Дальше добавляем `[CLS]` токен в начало последовательности

Дальше, подаем последовательность в модель и получаем предсказания для каждого токена. Отсекаем `[CLS]`, так как нам он не интересен.

In [ ]:
def tag_sentence(model, device, sentence, tokenizer, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        tokens = tokenizer.tokenize(sentence)
    else:
        tokens = sentence
    
    numericalized_tokens = tokenizer.convert_tokens_to_ids(tokens)
    numericalized_tokens = [text_field.init_token] + numericalized_tokens
        
    unk_idx = text_field.unk_token
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    predicted_tags = predicted_tags[1:]
        
    assert len(tokens) == len(predicted_tags)
    
    return tokens, predicted_tags, unks

Пример

In [ ]:
sentence = 'Павел Дуров сегодня анонсировал создание нового офиса в Санкт-Петербурге'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence,
                                  tokenizer,
                                  TEXT, 
                                  UD_TAGS)

print(unks)

In [ ]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")